In [3]:
import radosgw

In [38]:
access_key = '6LDLPN722I69N3AY9B95'
secret_key = 'nyPiW6pGu9sOEWlCTexOgXu54UMvMBm1zGzqyv3W'

rgwadmin = radosgw.connection.RadosGWAdminConnection(host='192.168.35.41', 
                                                     is_secure=False,
                                                     access_key=access_key,
                                                     secret_key=secret_key)



testuser1 = rgwadmin.get_user('mania')

bucket_info = rgwadmin.get_bucket('data', stats=True)
for key in bucket_info._object:
    print(key, '\t', bucket_info._object[key])

bucket 	 data
zonegroup 	 56960513-3595-4337-9d8a-8b6b68cd76bb
placement_rule 	 default-placement
explicit_placement 	 {'data_pool': '', 'data_extra_pool': '', 'index_pool': ''}
id 	 3ba5b0d9-ab69-488c-909d-7cb0237cafba.9286704.1
marker 	 3ba5b0d9-ab69-488c-909d-7cb0237cafba.9286704.1
index_type 	 Normal
owner 	 testuser
ver 	 0#644
master_ver 	 0#0
mtime 	 2020-04-26 14:10:26.621579
max_marker 	 0#
usage 	 {'rgw.main': {'size': 35271645920, 'size_actual': 35271720960, 'size_utilized': 35271645920, 'size_kb': 34444967, 'size_kb_actual': 34445040, 'size_kb_utilized': 34444967, 'num_objects': 36}}
bucket_quota 	 {'enabled': False, 'check_on_raw': True, 'max_size': -1, 'max_size_kb': 0, 'max_objects': -1}


In [185]:
import swiftclient
import json 

user = 'testuser:swift'
key = '7Xqb6gdsCE5Vu0clmk2qL0yjjy1NCNiFuaPlGQvJ'



def add_to_metadata_tree(metadata, name, size):
    path_element = name.split('/')
    
    meta_ptr = metadata
    for element in path_element:
        if element not in meta_ptr:
            meta_ptr[element] = {'objs': {}, 'size': 0 }
        meta_ptr[element]['size'] += size
        meta_ptr = meta_ptr[element]['objs']

def build_metadata_tree():    
    conn = swiftclient.Connection(
            user=user,
            key=key,
            authurl='http://192.168.35.41:80/auth/1.0')

    bucket_name = 'data'    
    metadata = {}
    
    metadata_swift =  conn.get_container(bucket_name)[1]
    for data in metadata_swift:
        full_name = data['name'];

        if full_name.endswith('_SUCCESS') : continue;
        add_to_metadata_tree(metadata, data['name'], data['bytes'])
        #break
        print(data['name'], data['bytes'])
    
build_metadata_tree()
#print(json.dumps(metadata, indent = 4))


HiBench/Aggregation/Input/uservisits/ 0
HiBench/Aggregation/Output/uservisits_aggre/000000_0 87
HiBench/Dfsioe/Input/_merged_reports.txt 470457
HiBench/Dfsioe/Input/io_control/in_file_test_io_0 112
HiBench/Dfsioe/Input/io_control/in_file_test_io_1 112
HiBench/Dfsioe/Input/io_control/in_file_test_io_10 113
HiBench/Dfsioe/Input/io_control/in_file_test_io_11 113
HiBench/Dfsioe/Input/io_control/in_file_test_io_12 113
HiBench/Dfsioe/Input/io_control/in_file_test_io_13 113
HiBench/Dfsioe/Input/io_control/in_file_test_io_14 113
HiBench/Dfsioe/Input/io_control/in_file_test_io_15 113
HiBench/Dfsioe/Input/io_control/in_file_test_io_16 113
HiBench/Dfsioe/Input/io_control/in_file_test_io_17 113
HiBench/Dfsioe/Input/io_control/in_file_test_io_18 113
HiBench/Dfsioe/Input/io_control/in_file_test_io_19 113
HiBench/Dfsioe/Input/io_control/in_file_test_io_2 112
HiBench/Dfsioe/Input/io_control/in_file_test_io_20 113
HiBench/Dfsioe/Input/io_control/in_file_test_io_21 113
HiBench/Dfsioe/Input/io_control/in

In [158]:
import math
import requests
import swiftclient
import json 


user = 'testuser:swift'
key = '7Xqb6gdsCE5Vu0clmk2qL0yjjy1NCNiFuaPlGQvJ'


def fetch_object(token, bucket_name, obj_name, ofs_s, ofs_e):
    print(bucket_name, obj_name)
    
    url = 'http://192.168.35.41:80/swift/v1/%s/%s'%(bucket_name, obj_name)
    headers = {"range":"bytes=%d-%d"%(ofs_s, ofs_e),
              "X-Auth-Token": token}
    
    print(url, headers)
    
    
    r=requests.get(url, headers=headers)
    
    print(len(r.content), ofs_e - ofs_s)
    '''
    local_p = subprocess.Popen(["curl -i http://%s:%d/swift/v1/%s/%s -X GET -H \"range: bytes=%d-%d\" -H \"KARIZ_PREFETCH: 1\" -H \"X-Auth-Token: %s\" -o /dev/null 2>&1 | tee %soutput-%s.txt" %(cfg.d3n_host, cfg.d3n_port, bucket_name, obj_name, s_off, e_off, token, './', obj_name)], shell=True)
    local_p.wait()
    '''



def prefetch_dataset(path, wave=-1, stride=0):
    # stride = -1 means prefetch the full object
    # wave = -1 means follow the partitions schema of original dataset
    cache_block_size = 4194304 # 4 MB
    
    path_element = path.split('/')
    
    meta_ptr = metadata
    for element in path_element:
        if element not in meta_ptr:
            return -1; # means could not prefetch this input
        meta_ptr = meta_ptr[element]['objs']
        
    for obj in meta_ptr:
        n_cache_blocks = meta_ptr[obj]['size']//cache_block_size
        
            
        ofs_s = (n_cache_blocks - stride)*cache_block_size if (stride) else 0 
        ofs_e = meta_ptr[obj]['size']
        
        fetch_object(token=token, bucket_name='data', obj_name=path+'/'+obj, ofs_s = ofs_s, ofs_e = ofs_e)
        
        # add to thread pool 
        
        
    
    #print(json.dumps(meta_ptr, indent = 4))

conn = swiftclient.Connection(
        user=user,
        key=key,
        authurl='http://192.168.35.41:80/auth/1.0')

token = conn.get_auth()[1]
print(token)
    
dpath = 'HiBench/Wordcount/Input'
prefetch_dataset(dpath, stride=176)

AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a
data HiBench/Wordcount/Input/part-m-00000
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00000 {'range': 'bytes=360710144-1102234211', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a'}
741524067 741524067
data HiBench/Wordcount/Input/part-m-00001
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00001 {'range': 'bytes=360710144-1102230641', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a'}
741520497 741520497
data HiBench/Wordcount/Input/part-m-00002
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00002 {'range': 'bytes=360710144-1102224903', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e1579032

741522071 741522071
data HiBench/Wordcount/Input/part-m-00026
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00026 {'range': 'bytes=360710144-1102245692', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a'}
741535548 741535548
data HiBench/Wordcount/Input/part-m-00027
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00027 {'range': 'bytes=360710144-1102239104', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a'}
741528960 741528960
data HiBench/Wordcount/Input/part-m-00028
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00028 {'range': 'bytes=360710144-1102241471', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a'}
741531327 741531327
data HiBench/Wordcount/Input/part-m

In [173]:
token = conn.get_auth()[1]
print(token)

def clear_cache():
    url = 'http://192.168.35.41:80/swift/v1/'
    headers = {"KARIZ_FLUSH_CACHE":"1",
              "X-Auth-Token": token}

    r=requests.delete(url, headers=headers)
    print(r)



#curl -i http://192.168.35.41:80/swift/v1/ -X DELETE -H "KARIZ_FLUSH_CACHE" -H "X-Auth-Token: $OS_AUTH_TOKEN"

AUTH_rgwtk0e00000074657374757365723a737769667401fa2c695db584693509ab5e1c8647367ad0222e648226b8451c455263b0451fa58d3b66
http://192.168.35.41:80/swift/v1/ {'KARIZ_FLUSH_CACHE': '1', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667401fa2c695db584693509ab5e1c8647367ad0222e648226b8451c455263b0451fa58d3b66'}
<Response [200]>


In [183]:
oids0 = '/local0/Kariz/expriments/microbenchmark/oids0'
oids1 = '/local0/Kariz/expriments/microbenchmark/oids1'

oids0_dict = {}
found = 0

with open(oids0, 'r') as fd0, open(oids1, 'r') as fd1:
    lns0 = fd0.readlines()
    lns1 = fd1.readlines()
    
    lns0 = [ln.split('__shadow_.')[1] for ln in lns0]
    lns1 = [ln.split('__shadow_.')[1] for ln in lns1]
    
    for oid in lns0:
        oids0_dict[oid] = 1;
        
    for oid in lns1:
        if oid in oids0_dict:
            found += 1;
    
    print(found, len(lns0), len(lns1), lns0[0], lns1[0])

0 8300 4096 44ZmSgPSdssfWV5nZAIiknKEH7mn0kS_1
 _3OPiXJZXMgLg-kubf3Pdbt5aohV4Yj_134

